# COGS 108 - Data Checkpoint

# Names

- Jake Heinlein
- Nathan Tripp
- Naomi Chin
- Leo Friedman
- Dante Tanjuatco

<a id='research_question'></a>
# Research Question

*Is the combination of an mlb free agents age and batting performance, measured by batting average and on base plus slugging percentage, indicative of their yearly salary, and if so, can an algorithm predict a players future contract based on these factors?*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

The first data set looks at MLB players stats by year, which we are using to look at players age, batting average, and on base plus slugging percentage. The second dataset shows free agent contracts, which we will use to find players yearly salary.

We are looking at players who are included in both sets, comparing batting performance along with age to their yearly salary.

# Setup

In [1]:

import pandas as pd
pd.set_option('display.max_rows', None)
import os

In [2]:
def salary_to_int(str_in):
    str_in = str_in.replace('$','')
    str_in = str_in.replace(',','')
    output = int(str_in)
    return output

def standardize_name(str_in):
    # str_in = str_in.replace(',', '')
    # str_in = remove_periods(str_in)
    name_list = str_in.split(', ')
    output = name_list[1] + ' ' + name_list[0]
    return output

def term_to_year(str_in):
    term_list = str(str_in).split('-')
    output = term_list[0]
    return output

def remove_periods(str_in):
    output = str_in.replace('.','')
    return output

In [3]:
# access the directory with MLB contracts
directory = 'data/contracts'
# initialize list of contract filenames
filenames = []
# concatenate contracts from 2011-2022
for filename in os.listdir(directory):
    filenames.append(str(directory) + "/" + filename)
contracts = pd.concat([pd.read_csv(f) for f in filenames])
# keep player, position, age, term, and AAV columns
contracts = contracts[["Player","Pos'n","Age","Term","AAV"]]
# remove pitcher information
contracts = contracts[contracts["Pos'n"].str.contains("hp") == False]
# remove any rows with empty data
contracts = contracts[contracts["AAV"].isna() == False]
# remove $1 contracts (0 year contracts)
contracts = contracts[contracts["AAV"] != "$1"]
# change AAV to an integer
contracts["AAV"] = contracts["AAV"].apply(salary_to_int)
# reset the indices
contracts = contracts.reset_index(drop = True)
# rename AAV (Average Annual Value) to Yearly Salary
contracts = contracts.rename({"AAV":"Yearly Salary"}, axis=1)
contracts["Player"] = contracts["Player"].apply(standardize_name)
contracts = contracts[["Player","Term","Yearly Salary"]]
contracts["Term"] = contracts["Term"].apply(term_to_year)
contracts = contracts.rename({"Term":"Year"}, axis = 1)
contracts = contracts.sort_values(by="Year")
print(contracts.iloc[1,1])
print(len(contracts["Player"].unique()))
display(contracts.head())

2011
339


,Player,Year,Yearly Salary
345,Jerry Hairston,2011,2000000
313,Adam Dunn,2011,14000000
312,Adrian Beltre,2011,16000000
311,Jayson Werth,2011,18000000
310,Carl Crawford,2011,20285714


In [4]:
directory = 'data/batting'
filenames = []
dataframes = []
for filename in os.listdir(directory):
    year = filename[:4]
    filepath = str(directory) + "/" + filename
    filenames.append(filepath)
    df = pd.read_csv(filepath)
    df['Year'] = year
    dataframes.append(df)
batting = pd.concat(dataframes)
batting = batting.sort_values(by = "Year")
batting = batting[["Player", "Pos", "Age", "AVG", "OPS", "Year"]]
batting = batting[batting["Player"].isin(contracts["Player"])]
# batting["Player"] = batting["Player"].apply(remove_periods)
# print(len(batting["Player"].unique()))
# print((contracts[contracts["Player"].isin(batting["Player"]) == False ])["Player"])
# for player in contracts["Player"].unique() :
#     if((batting["Player"].unique()).contains(player) == False):
#         print(player)
# print(contracts["Player"].unique().isin((batting["Player"].unique())))
# batting = batting[batting.isin(contracts["Player"])]
batting.head()
# batting["Player"].value_counts()

,Player,Pos,Age,AVG,OPS,Year
0,Ichiro Suzuki,OF,48,0.315,0.753,2010
448,Jorge Cantu,3B,40,0.235,0.606,2010
451,Mark DeRosa,OF,47,0.194,0.537,2010
457,Pat Burrell,OF,46,0.202,0.625,2010
458,Lucas Duda,1B,36,0.202,0.678,2010


# Data Cleaning

Describe your data cleaning steps here.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION